In [ ]:
#Python爬取视频之日本爱情电影
#https://blog.csdn.net/JosephPai/article/details/78897370
#需要外面的世界
# -*- coding:utf-8 -*-
import requests
from bs4 import BeautifulSoup
import os
import lxml
import time
import random
import re
import m3u8

class ViedeoCrawler():
    def __init__(self):
        self.url = "https://www.gavbus.com/"
        self.down_path = r"D:\Spider\DOWN"
        self.final_path = r"D:\Spider\FINAL"
        try:
            self.name = re.findall(r'/[A-Za-z]*-[0-9]*',self.url)[0][1:]
        except:
            self.name = "uncensord"
        self.headers = {
            'Connection': 'Keep-Alive',
            'Accept': 'text/html, application/xhtml+xml, */*',
            'Accept-Language': 'en-US,en;q=0.8,zh-Hans-CN;q=0.5,zh-Hans;q=0.3',
            'User-Agent':'Mozilla/5.0 (Linux; U; Android 6.0; zh-CN; MZ-m2 note Build/MRA58K) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/40.0.2214.89 MZBrowser/6.5.506 UWS/2.10.1.22 Mobile Safari/537.36'
        }

    def get_ip_list(self):
        print("正在获取代理列表...")
        url = 'http://www.xicidaili.com/nn/'
        html = requests.get(url=url, headers=self.headers).text
        soup = BeautifulSoup(html, 'lxml')
        ips = soup.find(id='ip_list').find_all('tr')
        ip_list = []
        for i in range(1, len(ips)):
            ip_info = ips[i]
            tds = ip_info.find_all('td')
            ip_list.append(tds[1].text + ':' + tds[2].text)
        print("代理列表抓取成功.")
        return ip_list

    def get_random_ip(self,ip_list):
        print("正在设置随机代理...")
        proxy_list = []
        for ip in ip_list:
            proxy_list.append('http://' + ip)
        proxy_ip = random.choice(proxy_list)
        proxies = {'http': proxy_ip}
        print("代理设置成功.")
        return proxies

    def get_uri_from_m3u8(self,realAdr):
        print("正在解析真实下载地址...")
        with open('temp.m3u8', 'wb') as file:
            file.write(requests.get(realAdr).content)
        m3u8Obj = m3u8.load('temp.m3u8')
        print("解析完成.")
        return m3u8Obj.segments

    def run(self):
        print("Start!")
        start_time = time.time()
        os.chdir(self.down_path)
        html = requests.get(self.url).text
        bsObj = BeautifulSoup(html, 'lxml')
        realAdr = bsObj.find(id="video-player").find("source")['src']

        # duration = bsObj.find('meta', {'property': "video:duration"})['content'].replace("\"", "")
        # limit = int(duration) // 10 + 3

        ip_list = self.get_ip_list()
        proxies = self.get_random_ip(ip_list)
        uriList = self.get_uri_from_m3u8(realAdr)
        i = 1   # count
        for key in uriList:
            if i%50==0:
                print("休眠10s")
                time.sleep(10)
            if i%120==0:
                print("更换代理IP")
                proxies = self.get_random_ip(ip_list)
            try:
                resp = requests.get(key.uri, headers = self.headers, proxies=proxies)
            except Exception as e:
                print(e)
                return
            if i < 10:
                name = ('clip00%d.ts' % i)
            elif i > 100:
                name = ('clip%d.ts' % i)
            else:
                name = ('clip0%d.ts' % i)
            with open(name,'wb') as f:
                f.write(resp.content)
                print('正在下载clip%d' % i)
            i = i+1
        print("下载完成！总共耗时 %d s" % (time.time()-start_time))
        print("接下来进行合并……")
        os.system('copy/b %s\\*.ts %s\\%s.ts' % (self.down_path,self.final_path, self.name))
        print("合并完成，请您欣赏！")
        y = input("请检查文件完整性，并确认是否要删除碎片源文件？(y/n)")
        if y=='y':
            files = os.listdir(self.down_path)
            for filena in files:
                del_file = self.down_path + '\\' + filena
                os.remove(del_file)
            print("碎片文件已经删除完成")
        else:
            print("不删除，程序结束。")

if __name__=='__main__':
    crawler = ViedeoCrawler()
    crawler.run()

In [55]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import json
import requests
import re
from scrapy.selector import Selector
 
def make_unicode(value, prefer_encodings=None):
    if prefer_encodings is None:
        prefer_encodings = ['utf8', 'gbk', 'gbk?']
    if isinstance(value, str) or value is None:
        return value
    if not isinstance(value, str):
        return value
    for enc in prefer_encodings:
        try:
            if enc.endswith('!'):
                return value.decode(enc[:-1], 'ignore')
            elif enc.endswith('?'):
                return value.decode(enc[:-1], 'replace')
            elif enc.endswith('&'):
                return value.decode(enc[:-1], 'xmlcharrefreplace')
            elif enc.endswith('\\'):
                return value.decode(enc[:-1], 'backslashreplace')
            else:
                return value.decode(enc)
        except UnicodeError:
            pass
    else:
        raise
 
def get_down_urls(url):
    response = requests.get(url, verify=False)
    response = Selector(text=make_unicode(response.text))
    alist = response.xpath('.//script')
    for href in alist:
        scripts_content = href.get()
        scripts_content = re.sub(r'[\n\t\r]', '', scripts_content)
        # print scripts_content
        if re.search('window.__init__', scripts_content):
            start_pos = scripts_content.index("{")
            end_pos = scripts_content.rindex("}")
            #print scripts_content[start_pos:end_pos + 1]
            json_str = json.loads(scripts_content[start_pos:end_pos + 1])
            cid = json_str['list'][0]['cid']
            title = json_str['title']
            print(title)
            aid = json_str['aid']
            #print(aid)
 
            donwlaod_url = "https://www.kanbilibili.com/api/video/%s/download?cid=%s&quality=112&page=1" % (aid, cid)
            get_download_urls = requests.get(donwlaod_url)
            get_download_urls = json.loads(get_download_urls.text)
            durls = get_download_urls['data']['durl']
            for durl in durls:
                down_url = durl['url']
                down_order = durl['order']
                #print ("down load url : %s " % down_url)
 
 
def Get_spalce_AVList(url):
    re_mid = re.search(r'\/([\d]+)\/',url)
    if re_mid:
        mid = re_mid.group(1)
    else:
        print ("url error can find mid in url")
        return
    aidList = []
    page = 1
    while True:
        url_vlist = "https://space.bilibili.com/ajax/member/getSubmitVideos?mid=%s&pagesize=100&tid=0&page=%s&keyword=&order=pubdate" % (mid, page)
        #print url_vlist
        response = requests.get(url_vlist, verify=False)
        vlist = json.loads(response.text)
        for info in vlist['data']['vlist']:
            aidList.append(info['aid'])
            #print info['aid']
        if vlist['data']['pages'] > page:
            page = page + 1
        else:
            break
    for aid in aidList:
        url_downLoad = "https://www.kanbilibili.com/video/av%s" % aid
        print (url_downLoad)
        get_down_urls(url_downLoad)
 
if __name__ == "__main__":
    space_url = "http://space.bilibili.com/73012391/video"
    Get_spalce_AVList(space_url)


C:\Users\Duke\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


https://www.kanbilibili.com/video/av38982821


C:\Users\Duke\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


高博视觉SLAM理论与实践（2018年第二期）
https://www.kanbilibili.com/video/av38892806


C:\Users\Duke\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

【Java从入门到精通】Java SE 第101-128讲
https://www.kanbilibili.com/video/av14453180


C:\Users\Duke\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


【Java从入门到精通】Java SE 第70-100讲
https://www.kanbilibili.com/video/av14452922


C:\Users\Duke\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


【Java从入门到精通】Java SE 第1-70讲


In [58]:
#!/usr/bin/env python
#coding:utf-8
 
import re
import base64

THUNDER_HEADER = "thunder://"
THUNDER_PREFIX = "AA"
THUNDER_SUFFIX = "ZZ"
ERROR = "ERROR!"

def is_url(func):
    def warpper(url):
        if re.match(r"(http|https|ftp|ed2k|thunder|qqdl)://",url):
            return func(url)
        else:
            return ERROR
    return warpper
def thunder2Real(url):
    url = url[len(THUNDER_HEADER):]
    url = url.encode("utf-8")
    url = base64.b64decode(url)
    url = url.decode("utf-8")
    url = url[len(THUNDER_PREFIX):-len(THUNDER_SUFFIX)]
    return url

 
#www.iplaypy.com
 
def test():
    url = 'thunder://QUFodHRwOi8vdXBvcy1oei1taXJyb3Jrb2RvdS5hY2d2aWRlby5jb20vdXBnY3hjb2RlLzU5LzUyLzY4Mjg1MjU5LzY4Mjg1MjU5LTEtODAuZmx2P2U9aWc4ZXV4Wk0yck5jTmJSZ2hiVVZob00xaGJOQmh3ZEV0bzhnNVgxMHVnTmNYQmxxTkNORXRvOEJUck52TkM3TVRYNTAyQzhmMmptTVFKNm1xRjJma2ExbXF4NmdxajBlTjBCNTk5TT0mZGVhZGxpbmU9MTU0OTQ0NTg1NiZnZW49cGxheXVybCZuYnM9MSZvaT0xNzAzMTc4Nzk0Jm9zPWtvZG91JnBsYXRmb3JtPWZsYXNoJnRyaWQ9Y2Y4MmY4MjM3OGVlNGI0MDg4MTRjNjU0ZDZlYzQ4MTQmdWlwaz01JnVwc2lnPTY1NTY4MTkxNjc2NTMxYzM1YmI1ODA3ZjUwZmVmNmU0Wlo=='
    #p = baseurl(sys.argv)
    p=thunder2Real(url)
    print('\n============请将下面地址复制到你的下载器中=============\n')
    print(p)
 
if __name__ == '__main__':
    test()


============请将下面地址复制到你的下载器中=============

http://upos-hz-mirrorkodou.acgvideo.com/upgcxcode/59/52/68285259/68285259-1-80.flv?e=ig8euxZM2rNcNbRghbUVhoM1hbNBhwdEto8g5X10ugNcXBlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M=&deadline=1549445856&gen=playurl&nbs=1&oi=1703178794&os=kodou&platform=flash&trid=cf82f82378ee4b408814c654d6ec4814&uipk=5&upsig=65568191676531c35bb5807f50fef6e4


In [46]:
import requests #http请求库
from pyquery import PyQuery #原生的CSS选择器


index_url = 'https://www.kanbilibili.com/video/av14808139'
response = requests.get(url=index_url).text


In [47]:
print(response)

<!DOCTYPE html>
<html>
<head>
  <title>【机器学习】 第6章_支持向量机SVM - 案例1：简单数据集分类 - ApacheCN v2.0.0 - 机器学习实战 下载(AV14808139)-演讲• 公开课-科技-看哔哩哔哩-bilibili日报|视频下载</title>
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<script type="text/javascript">
if (document.location.protocol != "https:") { 
  location.href = location.href.replace(/^http:/,"https:");
}
</script>
<link rel="icon" href="/img/kanbilibili.ico" type="image/x-icon" />
<link rel="shortcut icon" href="/img/kanbilibili.ico" type="image/x-icon" />
<meta charset="utf-8">

<meta name="keywords" content="演讲• 公开课,科技,av14808139,undefined,bilibili视频下载,B站视频下载,bilibili,看哔哩哔哩">


<meta name="description" content="【机器学习】 第6章_支持向量机SVM - 案例1：简单数据集分类 - ApacheCN v2.0.0 - 机器学习实战(MachineLearning In Action)">

<link rel="stylesheet" type="text/css" href="/css/main.css">

</head>
<body>
  <div id="app"><div class="video" data-reactroot="" data-reactid="1" data-react-checksum="1257837107"><div class="top-header" data-reactid="2"><div c

In [50]:
doc = PyQuery(response)
links = doc('head > script:nth-child(28)')

In [51]:
print(links)